# LightHearted: Tutorial

This notebook presents a walkthrough of how to use the LightHearted framework to create a lighting design. For this, we use the design employed in a concert with the Aarhus Symphony Orchestra (ASO) for a performance of Tchaikovsky's Romeo and Juliet Fantasy Overture. The script containing this can be found in the examples folder . By the end of this tutorial you should hopefully be familiar with the use of the five main objects used in a LightHearted workflow:

- FIFOBuffer - Used to contain and process ECG signals
- LightingArray - Used to define a lighting group and send them commands
- MappingArray - Used to contain and reduce data to be mapped from multiple ECG devices and derive spatial expansions that can be mapped to LightingArrays
- ContinuousMapper - Used to define the relationship between a MappingArray and a LightingArray
- TriggerMapper - Used to define event based triggers between FIFOBuffers and LightingArrays

## 1. Basics

### 1.1 Workflow

The general workflow for LightHearted is as follows:

1. ECG signals are transported over OSC and received in a script. For each signal, a first-in-first-out (FIFO) buffer is instanced.
2. Optional chains of transforms are applied to each FIFOBuffer, and placed into new instances of FIFOBuffers (e.g. filtering, QRS extraction).
3. Signal or transformed buffers can be assigned to instances of MappingArrays. These take multiple FIFOBuffers, with their position in the array corresponding to a spatial position. Each buffer is reduced to a single value using a chain of functions (e.g. mean, the newest value in the array).
4. The MappingArray can be used to generate spatial expansions, that is expanding the MappingArray to match the shape of a LightingArray. This can be done through a user-defined function (e.g. linear interpolation, filling the expansion with set values). A single MappingArray can generate multiple spatial expansions if the data is to be used to generate lighting across multiple lighting groups.
5. Groups of lighting fixtures are defined as LightingArray objects. If interpolations are to be used to generate spatial expansions in the MappingArray, anchor positions in the LightingArray can also be defined. These correspond to the spatial positions of the values in the MappingArray before expansion.
6. ContinuousMappers can be instanced to define the relationship between the MappingArrays/spatial expansions and the LightingArrays. These result in the intensity and colour parameters of the LightingArray being continuously updated.
7. The LightingArray can be used to send commands to the installed lighting system in the concert hall, sending the corresponding parameter messages to the fixtures.
8. TriggerMapper objects can be defined to generate action based mappings between FIFOBuffers. These take one buffer as a reference and another as a query (e.g. a signal buffer could be a reference, and a buffer of peaks indices could be a query). A chain of trigger functions can be defined, as long as the final function returns a bool (e.g. has a peak index in the peak buffer crossed a specified index in the signal buffer). Upon return of a True value from the trigger function chain, an action function is triggered.

These should be implemented in a single script, encapsulated in async functions. Familiarity with the asyncio library is assumed. If there are many parameters to define, it is also recommeded to make use of a config script, which is imported into the main script.

### 1.2 The ASO Design

The ASO design consists of three primary lighting groups and corresponding mappings:

1. A group of 14 horizonally organised LEDs reflected from the organ directly behind the stage. For this, we derive heart rate values from the ECG signals, interpolate the values to match the number of LEDs, and use these to drive shifts in RGB values within a defined colourmap. We also use the detection of an R-Peak to trigger changes in the lighting intensity.
2. A group of 36 baluster leds, mounted in the balconies around the audience. For these, we use the derived heart rate from the conductor's ECG to generate shifts in RGB within the colourmap.
3. A horizontally arranged row of 15 background LEDs, mounted in the wall to the rear of the stage. We use these to display the current colourmap.

A video of the design in action can be found [here](https://osf.io/c2zt9/?view_only=23cc8068eba347b2a7cc4f6dbc77adc3).

# 2. First Steps

## 2.1 Setting Up a Script

The first step is to create a new script, which we will call ASO25.py. At first, we will just import asyncio, used to define our async functions.

In [ ]:
import asyncio

We will then define a main function, and set it to run.

In [ ]:
async def main():
    pass

if __name__ == "__main__":
    asyncio.run(main())

We will define separate processing functions, and these will be launched from the main function. We will also initialise all of our LightHearted objects here.

The next step is to define identifiers for each of the ECG signals that we will use. It is important to note that these identifiers will be used across the design, e.g. as osc addresses, as dictionary keys, and data labels. For the purposes of this example, we will use four ECG signals - from the conductor, the french horn, the concertmaster, and an audience member. We will labels these ```"conductor"```, ```"brs"```, ```"vn1"```, and ```"aud"``` respectively. We will define them as a list ```osc_addresses```, as there first use will be in the reception of the ECG data over OSC. We can place this line in the main function.

In [ ]:
osc_addresses = ["/aud", "/brs", "/conductor", "/vn1"]

## 2.2 Getting Data Into the Script

The next step is to start receiving data in the script. We can either receive data from four ECG devices over OSC, or we can use the ```csv_simulator``` to read recorded ECG csv files in realtime. We will walk through both of these.

### 2.2.1 Sending Data from an ECG Sensor

LightHearted is intented to be device agnostic, that is that it can function with any ECG device. In view of this, ECG data is expected to be received over OSC messages. Currently, LightHearted contains inbuilt support for devices from [SIFILabs](https://sifilabs.com/). This can be accessed through running the script ```sifilabs.py``` in the ```acquisition``` module. This will connect to and stream an arbritrary number of SIFILabs devices over OSC.

The script is parameterised in ```sificonfig.py```. The most important variable here is the ```mac_dict```, which provides the device identifier and MAC address. It is important to note that the keys in the ```mac_dict``` should match the OSC addresses set in the LightHearted design. So in our case, we would parameterise it as so:

In [ ]:
mac_dict = { 
    "/aud": "MACADDRESS1",
    "/brs": "MACADDRESS2",
    "/conductor": "MACADDRESS3",
    "/vn1": "MACADDRESS4"
    }

The IP and port of the server receiving the data (in our case to be used in our design) are also set here in the ```sifi_receiver_ip``` and ```sifi_receiver_port``` variables.

### 2.2.2 Reading a CSV File in Realtime

LightHearted also supports the use of previously recorded ECG data. This is done through the ```csv_simulator``` module. All the csv files should be placed in a single directory, and importantly, their names should match the OSC addresses specified above. In our case, our four csv files should be named:

- ```aud.csv```
- ```brs.csv```
- ```conductor.csv```
- ```vn1.csv```

The reader is parameterised through ```sim_config.py```. The ```filepath``` variable specifies the directory in which the csv files are located. If ```None``` is provided, this defaults to the ```csv``` directory. The ```column``` specifies the column of the csv to read from. The ```csv_sr``` set the speed of playback.

Both of these methods can be integrated into a LightHearted script by calling them as a process within the script. In our case, we will use the csv_reader and define a function where it can run on the press of the ```'r'``` key.

In [ ]:
import aioconsole
from multiprocessing import Process
from csv_simulator.csv_simulator import csv_sim

async def listen_for_commands() -> None:
    """
    Asynchronously listens for user commands from the console.
    - 'q': Cancels all running tasks and stops the event loop (quits the program).
    - 'r': Starts the CSV simulator in a separate process.

    Parameters
    ----------
    None
    """
    while True:
        user_input = await aioconsole.ainput("Enter 'r' to run simulator: ")
        if user_input.lower() == 'r':
            p_csv = Process(target=csv_sim)
            p_csv.start()

In our main block, we now define a task for the listener.

In [ ]:
async def main():

    task_commands = asyncio.create_task(listen_for_commands())

We will now be able to launch the csv reader on the press of the ```'r'``` key.

### 2.2.3 Receiving Data in the Script

So how do we receive this data in our script? There are several inbuilt functions to setup an osc server and process incoming data. However, first we need to initialise a location for the data to be placed. This is where we are introduced to the first key object for a LightHearted design - the ```FIFOBuffer``` object. This is a buffer of a fixed size, which operates on a first-in-first-out principle, meaning that as data is added to the buffer that exceeds the buffer length, the oldest data in the buffer is deleted. We instantiate the buffer with a single argument, the size of the buffer.

In [9]:
import sys
sys.path.append("../")
from acquisition.fifo_buffer import FIFOBuffer

buffer = FIFOBuffer(10)

We can add data to the buffer using the ```enqueue``` method. ```Int```, ```Float```, ```List```, ```Tuple``` and ```np.ndarray``` are accepted as valid to enqueue. Lists, tuples, and numpy arrays are flattened before they are enqueued. The ```get_buffer``` method returns the current buffer.

In [10]:
# int
buffer.enqueue(1)
print(f"int: {buffer.get_buffer()}")
# float
buffer.enqueue(1.5)
print(f"float: {buffer.get_buffer()}")
# list
buffer.enqueue([2, 3, 4])
print(f"list: {buffer.get_buffer()}")
# tuple
buffer.enqueue((5, 6))
print(f"tuple: {buffer.get_buffer()}")
# np.ndarray
import numpy as np
buffer.enqueue(np.array([7, 8, 9]))
print(f"np.ndarray: {buffer.get_buffer()}")

# first in, first out
buffer.enqueue(10)
print(f"FIFO (1 is removed, 10 is added): {buffer.get_buffer()}")

int: [1.]
float: [1.  1.5]
list: [1.  1.5 2.  3.  4. ]
tuple: [1.  1.5 2.  3.  4.  5.  6. ]
np.ndarray: [1.  1.5 2.  3.  4.  5.  6.  7.  8.  9. ]
FIFO (1 is removed, 10 is added): [ 1.5  2.   3.   4.   5.   6.   7.   8.   9.  10. ]


The buffer can be cleared with the ```clear_buffer``` method.

In [11]:
buffer.clear_buffer()
print(f"Buffer after clearing: {buffer.get_buffer()}")

Buffer after clearing: []


The ```set_buffer``` method clears and sets the buffer to the current values. The ```resize_buffer``` argument can reset the fixed length of the buffer to the new input. ```get_size``` returns the current size of the buffer and ```get_max_size``` returns the fixed length. ```is_full``` returns a boolean if the the buffer has reached its fixed length.

In [12]:
buffer.enqueue(np.arange(10))
print(f"Buffer after enqueing: {buffer.get_buffer()}")
print("\n")

buffer.set_buffer(np.arange(5), resize_buffer=False)
print(f"Buffer after setting: {buffer.get_buffer()}")
print(f"Buffer size: {buffer.get_size()}")
print(f"Buffer max size: {buffer.get_max_size()}")
print(f"Is buffer full? {buffer.is_full()}")
print("\n")

buffer.set_buffer(np.arange(5), resize_buffer=True)
print(f"Buffer after setting with resize: {buffer.get_buffer()}")
print(f"Buffer size after resize: {buffer.get_size()}")
print(f"Buffer max size after resize: {buffer.get_max_size()}")
print(f"Is buffer full after resize? {buffer.is_full()}")

Buffer after enqueing: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


Buffer after setting: [0. 1. 2. 3. 4.]
Buffer size: 5
Buffer max size: 10
Is buffer full? False


Buffer after setting with resize: [0. 1. 2. 3. 4.]
Buffer size after resize: 5
Buffer max size after resize: 5
Is buffer full after resize? True
